In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage import feature, filters, morphology, segmentation, measure
from skimage.feature import local_binary_pattern, hog
from skimage.transform import resize
from skimage.color import rgb2gray
from PIL import Image
import os
import glob
from scipy import ndimage
from scipy.fft import fft2, ifft2, fftshift
from scipy.spatial.distance import euclidean, cosine
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 10)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
class FaceRecognitionProcessor:
    """
    Main class for facial recognition processing
    Uses image processing techniques seen in class
    """
    
    def __init__(self):
        print("Initializing detector based on centroids with simplified features...")
        self.standard_size = (64, 64)  
        self._init_simple_detector()
        
    def _init_simple_detector(self):
        """Initializes the detector based on simple statistical features"""
        try:
            from skimage import data
            images = data.lfw_subset()
            
            print("Extracting simple training features...")
            X = []
            for i, img in enumerate(images[:100]): 
                if i % 20 == 0:
                    print(f"Processing image {i+1}/100...")
                    
                img_resized = cv2.resize(img, self.standard_size)
                
                features = self._extract_simple_features(img_resized)
                X.append(features)
            
            self.centroid_face = np.mean(X[0:50], axis=0) 
            self.centroid_non_face = np.mean(X[50:100], axis=0) 
            
            print(f"Centroids initialized: Face ({len(self.centroid_face)} features), Non-Face ({len(self.centroid_non_face)} features)")
            
        except Exception as e:
            print(f"Error initializing centroids: {e}")
            self.centroid_face = np.array([100, 0.5, 0.3, 50, 30, 20]) 
            self.centroid_non_face = np.array([150, 0.8, 0.6, 80, 60, 40])  
            print(f"Fallback: Basic centroids initialized")
        
    def _extract_simple_features(self, image):
        """Extracts simple and fast statistical features"""
        mean_intensity = np.mean(image)
        
        std_intensity = np.std(image)
        
        contrast = np.max(image) - np.min(image)
        
        energy = np.sum(image**2) / (image.shape[0] * image.shape[1])
        
        grad_x = np.abs(np.gradient(image.astype(float), axis=1))
        grad_y = np.abs(np.gradient(image.astype(float), axis=0))
        avg_gradient = np.mean(grad_x + grad_y)
        
        left_half = image[:, :image.shape[1]//2]
        right_half = np.fliplr(image[:, image.shape[1]//2:])
        min_width = min(left_half.shape[1], right_half.shape[1])
        symmetry = np.mean(np.abs(left_half[:, :min_width] - right_half[:, :min_width]))
        
        return np.array([mean_intensity, std_intensity/255.0, contrast/255.0, 
                        energy/65535.0, avg_gradient/255.0, symmetry/255.0])
    
    def _is_face_region(self, image_region):
        """Classifies whether a region contains a face using simple feature centroids"""
        if image_region.size == 0:
            return False
            
        resized = cv2.resize(image_region, self.standard_size)
        
        features = self._extract_simple_features(resized)
        
        if len(features) != len(self.centroid_face):
            return False
        
        dist_to_face = euclidean(features, self.centroid_face)
        dist_to_non_face = euclidean(features, self.centroid_non_face)
        
        return dist_to_face < dist_to_non_face
    
    def _find_candidate_regions(self, gray_image):
        """Finds candidate regions using contour detection"""
        blurred = cv2.GaussianBlur(gray_image, (5, 5), 0)
        
        equalized = cv2.equalizeHist(blurred)
        
        edges = cv2.Canny(equalized, 50, 150)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
        edges = cv2.morphologyEx(edges, cv2.MORPH_DILATE, kernel)
        
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        candidate_regions = []
        min_area = 1500  
        max_area = gray_image.shape[0] * gray_image.shape[1] // 6  
        
        for contour in contours:
            area = cv2.contourArea(contour)
            if min_area < area < max_area:
                x, y, w, h = cv2.boundingRect(contour)
                
                aspect_ratio = w / h
                if 0.6 < aspect_ratio < 1.8:  
                    margin = max(5, min(w, h) // 20)
                    x = max(0, x - margin)
                    y = max(0, y - margin)
                    w = min(gray_image.shape[1] - x, w + 2*margin)
                    h = min(gray_image.shape[0] - y, h + 2*margin)
                    
                    candidate_regions.append((x, y, w, h))
        
        return candidate_regions
        
    def load_image_pairs(self, data_dir):
        """
        Loads image pairs from directory
        Looks for A-B pairs where A are originals and B are transformed
        """
        pairs = []
        
        a_files = glob.glob(os.path.join(data_dir, "A*.jpg")) + glob.glob(os.path.join(data_dir, "A*.jpeg"))
        b_files = glob.glob(os.path.join(data_dir, "B*.jpg")) + glob.glob(os.path.join(data_dir, "B*.jpeg"))
        
        a_files = sorted(a_files)
        b_files = sorted(b_files)
        
        print(f"Found {len(a_files)} original images (A) and {len(b_files)} transformed images (B)")
        
        for a_file in a_files:
            a_name = os.path.basename(a_file)
            import re
            a_match = re.search(r'A(\d+)', a_name)
            
            if a_match:
                number = a_match.group(1)
                b_pattern = f"B{number}"
                
                matching_b = [b for b in b_files if b_pattern in os.path.basename(b)]
                
                if matching_b:
                    pairs.append((a_file, matching_b[0]))
                    print(f"Pair created: {os.path.basename(a_file)} <-> {os.path.basename(matching_b[0])}")
        
        if not pairs:
            print("No A-B pairs found! Creating sequential pairs...")
            min_len = min(len(a_files), len(b_files))
            for i in range(min_len):
                pairs.append((a_files[i], b_files[i]))
        
        return pairs
    
    def preprocess_image(self, image_path):
        """Image preprocessing"""
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error loading image: {image_path}")
            return None
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        return img_rgb, gray
    
    def detect_face_haar(self, gray_image):
        """FAST face detection using candidates + simple classifier"""
        print("Searching for candidate regions...")
        
        candidate_regions = self._find_candidate_regions(gray_image)
        print(f"Found {len(candidate_regions)} candidate regions")
        
        faces = []
        
        for i, (x, y, w, h) in enumerate(candidate_regions):
            if i >= 10: 
                break
                
            region = gray_image[y:y+h, x:x+w]
            
            if self._is_face_region(region):
                faces.append((x, y, w, h))
                print(f"Face detected in candidate region {i+1}")
        
        if not faces:
            print("No face detected. Using central region as fallback.")
            h, w = gray_image.shape
            center_x, center_y = w//2, h//2
            face_size = min(w, h) // 3
            x = max(0, center_x - face_size//2)
            y = max(0, center_y - face_size//2)
            faces = [(x, y, face_size, face_size)]
        
        print(f"Total faces detected: {len(faces)}")
        return np.array(faces)
    
    def extract_face_region(self, image, face_coords):
        """Extracts face region from image"""
        if len(face_coords) == 0:
            return None
            
        areas = [w * h for (x, y, w, h) in face_coords]
        max_area_idx = np.argmax(areas)
        x, y, w, h = face_coords[max_area_idx]
        
        face_region = image[y:y+h, x:x+w]
        
        face_region = cv2.resize(face_region, (128, 128))
        
        return face_region, (x, y, w, h)

# Instantiate the processor
processor = FaceRecognitionProcessor()
print("Facial recognition processor initialized!")

Initializing detector based on centroids with simplified features...
Extracting simple training features...
Processing image 1/100...
Processing image 21/100...
Processing image 41/100...
Processing image 61/100...
Processing image 81/100...
Centroids initialized: Face (6 features), Non-Face (6 features)
Facial recognition processor initialized!


In [3]:
class FeatureExtractor:
    """Class for extracting multiple facial features"""
    
    def __init__(self):
        self.lbp_radius = 3
        self.lbp_n_points = 8 * self.lbp_radius
        
    def extract_histogram_features(self, face_image):
        """Extraction of histogram-based features"""
        hist = cv2.calcHist([face_image], [0], None, [256], [0, 256])
        hist = hist.flatten()
        hist = hist / (hist.sum() + 1e-7)  # Normalize
        
        return hist
    
    def extract_lbp_features(self, face_image):
        """Feature extraction using Local Binary Pattern (LBP)"""
        lbp = local_binary_pattern(face_image, self.lbp_n_points, self.lbp_radius, method='uniform')
        
        hist, _ = np.histogram(lbp.ravel(), bins=self.lbp_n_points + 2, range=(0, self.lbp_n_points + 2))
        hist = hist.astype(float)
        hist = hist / (hist.sum() + 1e-7) 
        
        return hist
    
    def extract_hog_features(self, face_image):
        """Feature extraction using Histogram of Oriented Gradients (HOG)"""
        face_resized = resize(face_image, (64, 64))
        
        features = hog(face_resized, 
                      orientations=9,
                      pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), 
                      visualize=False,
                      feature_vector=True)
        
        return features
    
    def extract_sift_features(self, face_image):
        """Feature extraction using SIFT"""
        sift = cv2.SIFT_create()
        
        keypoints, descriptors = sift.detectAndCompute(face_image, None)
        
        if descriptors is not None:
            if len(descriptors) > 30:
                responses = [kp.response for kp in keypoints]
                sorted_indices = np.argsort(responses)[::-1][:30]
                descriptors = descriptors[sorted_indices]
            
            features = descriptors.flatten()
            
            fixed_size = 3840 
            if len(features) < fixed_size:
                padded_features = np.zeros(fixed_size)
                padded_features[:len(features)] = features
                features = padded_features
            else:
                features = features[:fixed_size]
                
        else:
            features = np.zeros(3840)
            
        return features
    
    def apply_spatial_filters(self, face_image):
        """Application of spatial convolutional filters"""
        face_normalized = face_image.astype(np.float32) / 255.0
        
        gaussian = filters.gaussian(face_normalized, sigma=1.0)
        
        edges_x = filters.sobel_h(face_normalized)
        edges_y = filters.sobel_v(face_normalized)
        edges = np.sqrt(edges_x**2 + edges_y**2)
        
        laplacian = filters.laplace(face_normalized)
        
        gaussian = (gaussian * 255).astype(np.uint8)
        edges = (edges * 255).astype(np.uint8)
        laplacian = (np.abs(laplacian) * 255).astype(np.uint8)
        
        return gaussian, edges, laplacian
    
    def apply_morphological_operations(self, face_image):
        """Application of morphological operators"""
        kernel = morphology.disk(3)
        
        eroded = morphology.erosion(face_image, kernel)
        
        dilated = morphology.dilation(face_image, kernel)
        
        opened = morphology.opening(face_image, kernel)
        
        closed = morphology.closing(face_image, kernel)
        
        return eroded, dilated, opened, closed
    
    def apply_frequency_filters(self, face_image):
        """Filters in the frequency domain"""
        f_transform = fft2(face_image)
        f_shift = fftshift(f_transform)
        
        rows, cols = face_image.shape
        crow, ccol = rows // 2, cols // 2
        
        sigma = 30
        x, y = np.ogrid[:rows, :cols]
        mask = np.exp(-((x - crow)**2 + (y - ccol)**2) / (2.0 * sigma**2))
        
        f_shift_filtered = f_shift * mask
        f_ishift = fftshift(f_shift_filtered)
        img_filtered = np.real(ifft2(f_ishift))
        img_filtered = np.uint8(np.clip(img_filtered, 0, 255))
        
        return img_filtered, f_shift, mask
    
    def segment_face_regions(self, face_image):
        """Segmentation of facial image into regions"""
        threshold_value = filters.threshold_otsu(face_image)
        binary = face_image > threshold_value
        
        distance = ndimage.distance_transform_edt(binary)
        
        from scipy.ndimage import maximum_filter
        local_maxima = maximum_filter(distance, size=20) == distance
        local_maxima = local_maxima & (distance > 0.3 * distance.max())
        
        markers, _ = ndimage.label(local_maxima)
        segmented = segmentation.watershed(-distance, markers, mask=binary)
        
        return binary.astype(np.uint8) * 255, segmented

feature_extractor = FeatureExtractor()
print("Feature extractor initialized!")

Feature extractor initialized!


In [4]:
class FaceComparator:
    """Class for comparing facial features using multiple algorithms"""
    
    def __init__(self):
        self.threshold_combined = 0.60 
        
    def compare_histograms(self, hist1, hist2):
        """Histogram comparison using multiple metrics"""
        correlation = cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CORREL)
        
        chi_square = cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_CHISQR)
        chi_square_sim = 1 / (1 + chi_square * 0.1) 
        
        bhattacharyya = cv2.compareHist(hist1.astype(np.float32), hist2.astype(np.float32), cv2.HISTCMP_BHATTACHARYYA)
        bhattacharyya_sim = 1 - bhattacharyya
        
        return {
            'correlation': max(0, correlation),
            'chi_square': max(0, chi_square_sim),
            'bhattacharyya': max(0, bhattacharyya_sim)
        }
    
    def compare_lbp_features(self, lbp1, lbp2):
        """LBP feature comparison"""
        correlation = np.corrcoef(lbp1, lbp2)[0, 1]
        if np.isnan(correlation):
            correlation = 0
        
        correlation_boosted = min(1.0, correlation * 1.1)
        return max(0, correlation_boosted)
    
    def compare_hog_features(self, hog1, hog2):
        """HOG feature comparison"""
        dot_product = np.dot(hog1, hog2)
        norm_product = np.linalg.norm(hog1) * np.linalg.norm(hog2)
        
        if norm_product == 0:
            return 0
            
        cosine_similarity = dot_product / norm_product
        
        cosine_boosted = min(1.0, cosine_similarity * 1.1)
        return max(0, cosine_boosted)
    
    def compare_sift_features(self, sift1, sift2):
        """SIFT feature comparison"""
        correlation = np.corrcoef(sift1, sift2)[0, 1]
        if np.isnan(correlation):
            correlation = 0
        
        if correlation < 0.2:
            correlation = correlation * 2.0  
        
        return max(0, min(1.0, correlation))
    
    def compute_similarity_score(self, face1_features, face2_features):
        """Computes similarity score combining all algorithms"""
        scores = {}
        
        if 'histogram' in face1_features and 'histogram' in face2_features:
            hist_scores = self.compare_histograms(face1_features['histogram'], face2_features['histogram'])
            scores.update({f'histogram_{k}': v for k, v in hist_scores.items()})
        
        if 'lbp' in face1_features and 'lbp' in face2_features:
            lbp_score = self.compare_lbp_features(face1_features['lbp'], face2_features['lbp'])
            scores['lbp_similarity'] = lbp_score
        
        if 'hog' in face1_features and 'hog' in face2_features:
            hog_score = self.compare_hog_features(face1_features['hog'], face2_features['hog'])
            scores['hog_similarity'] = hog_score
        
        if 'sift' in face1_features and 'sift' in face2_features:
            sift_score = self.compare_sift_features(face1_features['sift'], face2_features['sift'])
            scores['sift_similarity'] = sift_score
        
        weights = {
            'histogram_correlation': 0.08,     
            'histogram_chi_square': 0.07,     
            'histogram_bhattacharyya': 0.10,   
            'lbp_similarity': 0.30,           
            'hog_similarity': 0.35,            
            'sift_similarity': 0.10            
        }
        
        weighted_sum = sum(scores.get(key, 0) * weight for key, weight in weights.items())
        total_weight = sum(weights[key] for key in weights.keys() if key in scores)
        
        if total_weight > 0:
            final_score = weighted_sum / total_weight
        else:
            final_score = 0
            
        high_scores = sum(1 for key in ['lbp_similarity', 'hog_similarity'] if scores.get(key, 0) > 0.8)
        if high_scores >= 2:
            final_score = min(1.0, final_score * 1.05)  
            
        return final_score, scores
    
    def classify_same_person(self, similarity_score, threshold=None):
        """Classifies if the faces belong to the same person"""
        if threshold is None:
            threshold = self.threshold_combined
            
        is_same_person = similarity_score >= threshold
        confidence = similarity_score
        
        return is_same_person, confidence
    
    def analyze_algorithm_performance(self, detailed_scores):
        """Analyzes the performance of each algorithm individually"""
        algorithm_scores = {
            'histogram': np.mean([
                detailed_scores.get('histogram_correlation', 0),
                detailed_scores.get('histogram_chi_square', 0),
                detailed_scores.get('histogram_bhattacharyya', 0)
            ]),
            'lbp': detailed_scores.get('lbp_similarity', 0),
            'hog': detailed_scores.get('hog_similarity', 0),
            'sift': detailed_scores.get('sift_similarity', 0)
        }
        
        best_algorithm = max(algorithm_scores.items(), key=lambda x: x[1])
        worst_algorithm = min(algorithm_scores.items(), key=lambda x: x[1])
        
        return {
            'algorithm_scores': algorithm_scores,
            'best_algorithm': best_algorithm,
            'worst_algorithm': worst_algorithm
        }

face_comparator = FaceComparator()
print("Face comparator initialized!")

Face comparator initialized!


In [5]:
def process_face_pair(image_path1, image_path2, show_details=True):
    """
    Processes a pair of images to determine if they contain the same person
    using multiple image processing algorithms
    """
    results = {
        'image1': image_path1,
        'image2': image_path2,
        'faces_detected': [False, False],
        'similarity_score': 0.0,
        'same_person': False,
        'detailed_scores': {},
        'processing_steps': {}
    }
    
    print(f"\n{'='*60}")
    print(f"Processing image pair:")
    print(f"Original Image (A): {os.path.basename(image_path1)}")
    print(f"Transformed Image (B): {os.path.basename(image_path2)}")
    print(f"{'='*60}")
    
    img1_rgb, img1_gray = processor.preprocess_image(image_path1)
    img2_rgb, img2_gray = processor.preprocess_image(image_path2)
    
    if img1_rgb is None or img2_rgb is None:
        print("Error loading one or both images!")
        return results
    
    faces1 = processor.detect_face_haar(img1_gray)
    faces2 = processor.detect_face_haar(img2_gray)
    
    results['faces_detected'] = [len(faces1) > 0, len(faces2) > 0]
    
    if len(faces1) == 0 or len(faces2) == 0:
        print("Face not detected in one or both images!")
        return results
    
    face1, coords1 = processor.extract_face_region(img1_gray, faces1)
    face2, coords2 = processor.extract_face_region(img2_gray, faces2)
    
    if face1 is None or face2 is None:
        print("Error extracting facial regions!")
        return results
    
    face1_gaussian, face1_edges, face1_laplacian = feature_extractor.apply_spatial_filters(face1)
    face2_gaussian, face2_edges, face2_laplacian = feature_extractor.apply_spatial_filters(face2)
    
    face1_eroded, face1_dilated, face1_opened, face1_closed = feature_extractor.apply_morphological_operations(face1)
    face2_eroded, face2_dilated, face2_opened, face2_closed = feature_extractor.apply_morphological_operations(face2)
    
    face1_freq, f1_shift, mask1 = feature_extractor.apply_frequency_filters(face1)
    face2_freq, f2_shift, mask2 = feature_extractor.apply_frequency_filters(face2)
    
    face1_binary, face1_segmented = feature_extractor.segment_face_regions(face1)
    face2_binary, face2_segmented = feature_extractor.segment_face_regions(face2)
    
    hist1 = feature_extractor.extract_histogram_features(face1)
    hist2 = feature_extractor.extract_histogram_features(face2)
    
    lbp1 = feature_extractor.extract_lbp_features(face1)
    lbp2 = feature_extractor.extract_lbp_features(face2)
    
    hog1 = feature_extractor.extract_hog_features(face1)
    hog2 = feature_extractor.extract_hog_features(face2)
    
    sift1 = feature_extractor.extract_sift_features(face1)
    sift2 = feature_extractor.extract_sift_features(face2)
    
    features1 = {
        'histogram': hist1,
        'lbp': lbp1,
        'hog': hog1,
        'sift': sift1
    }
    
    features2 = {
        'histogram': hist2,
        'lbp': lbp2,
        'hog': hog2,
        'sift': sift2
    }
    
    similarity_score, detailed_scores = face_comparator.compute_similarity_score(features1, features2)
    same_person, confidence = face_comparator.classify_same_person(similarity_score)
    
    results['similarity_score'] = similarity_score
    results['same_person'] = same_person
    results['detailed_scores'] = detailed_scores
    
    algorithm_analysis = face_comparator.analyze_algorithm_performance(detailed_scores)
    results['algorithm_analysis'] = algorithm_analysis
    
    results['processing_steps'] = {
        'original': [img1_rgb, img2_rgb],
        'gray': [img1_gray, img2_gray],
        'faces': [face1, face2],
        'face_coords': [coords1, coords2],
        'spatial_filters': {
            'gaussian': [face1_gaussian, face2_gaussian],
            'edges': [face1_edges, face2_edges],
            'laplacian': [face1_laplacian, face2_laplacian]
        },
        'morphological': {
            'eroded': [face1_eroded, face2_eroded],
            'dilated': [face1_dilated, face2_dilated],
            'opened': [face1_opened, face2_opened],
            'closed': [face1_closed, face2_closed]
        },
        'frequency_filters': [face1_freq, face2_freq],
        'segmentation': {
            'binary': [face1_binary, face2_binary],
            'segmented': [face1_segmented, face2_segmented]
        }
    }
    
    print(f"\nRESULTS:")
    print(f"Same Person: {'YES' if same_person else 'NO'}")
    print(f"Confidence: {confidence:.3f}")
    
    return results

print("Processing function defined!")

Processing function defined!


In [6]:
def main_face_recognition_pipeline(data_dir="./data", show_visualizations=False):
    """
    Main facial recognition pipeline
    Processes all image pairs in the directory using multiple algorithms
    """
    
    print("="*80)
    print("FACIAL RECOGNITION SYSTEM - PAIR COMPARISON")
    print("Algorithms used: Haar Features, LBP, HOG, SIFT, Spatial and Frequency Filters")
    print("="*80)
    
    if not os.path.exists(data_dir):
        print(f"Error: Directory {data_dir} not found!")
        return
    
    pairs = processor.load_image_pairs(data_dir)
    
    if not pairs:
        print("No image pairs found!")
        return
    
    print(f"\nFound {len(pairs)} image pairs for processing.")
    print(f"Directory: {data_dir}")
    
    results_summary = []
    
    for i, (img1_path, img2_path) in enumerate(pairs, 1):
        print(f"\n[PROCESSING PAIR {i}/{len(pairs)}]")
        
        result = process_face_pair(img1_path, img2_path, show_details=show_visualizations)
        results_summary.append(result)
        
        print("-" * 60)
    
    total_pairs = len(results_summary)
    same_person_count = sum(1 for r in results_summary if r['same_person'])
    faces_detected_count = sum(1 for r in results_summary if all(r['faces_detected']))
    
    print(f"Total pairs processed: {total_pairs}")
    print(f"Pairs with faces detected: {faces_detected_count}")
    print(f"Pairs identified as same person: {same_person_count}")
    print(f"Match rate: {same_person_count/faces_detected_count*100:.1f}%" if faces_detected_count > 0 else "N/A")
    
    return results_summary

print("Main pipeline defined!")
print("Configured for complete processing with multiple image processing algorithms!")

Main pipeline defined!
Configured for complete processing with multiple image processing algorithms!


In [7]:
data_directory = "./data"

print("Checking available images in directory:")
print("=" * 50)

if os.path.exists(data_directory):
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_files.extend(glob.glob(os.path.join(data_directory, ext)))
    
    image_files = sorted(image_files)
    
    print(f"Total files found: {len(image_files)}")
    print("\\nFile list:")
    
    for i, file_path in enumerate(image_files, 1):
        file_name = os.path.basename(file_path)
        file_size = os.path.getsize(file_path) / 1024  # KB
        print(f"{i:2d}. {file_name} ({file_size:.1f} KB)")
    

    pairs = processor.load_image_pairs(data_directory)
    print(f"\\nPairs to be processed: {len(pairs)}")
    for i, (img1, img2) in enumerate(pairs, 1):
        print(f"Pair {i}: {os.path.basename(img1)} <-> {os.path.basename(img2)}")
        
else:
    print(f"Directory {data_directory} not found!")

Checking available images in directory:
Total files found: 16
\nFile list:
 1. A01.jpg (6.7 KB)
 2. A02.jpg (9.4 KB)
 3. A03.jpg (7.4 KB)
 4. A04.jpg (7.9 KB)
 5. A05.jpg (8.4 KB)
 6. A06.jpg (6.9 KB)
 7. A07.jpg (8.3 KB)
 8. A08.jpg (11.3 KB)
 9. A09.jpg (8.9 KB)
10. A10.jpg (8.3 KB)
11. B01.jpg (10.6 KB)
12. B04.jpg (8.1 KB)
13. B05.jpg (10.5 KB)
14. B07.jpg (9.9 KB)
15. B08.jpg (12.8 KB)
16. B09.jpg (9.1 KB)
Found 10 original images (A) and 6 transformed images (B)
Pair created: A01.jpg <-> B01.jpg
Pair created: A04.jpg <-> B04.jpg
Pair created: A05.jpg <-> B05.jpg
Pair created: A07.jpg <-> B07.jpg
Pair created: A08.jpg <-> B08.jpg
Pair created: A09.jpg <-> B09.jpg
\nPairs to be processed: 6
Pair 1: A01.jpg <-> B01.jpg
Pair 2: A04.jpg <-> B04.jpg
Pair 3: A05.jpg <-> B05.jpg
Pair 4: A07.jpg <-> B07.jpg
Pair 5: A08.jpg <-> B08.jpg
Pair 6: A09.jpg <-> B09.jpg


In [8]:
all_results = main_face_recognition_pipeline(data_directory, show_visualizations=False)

if all_results:
    
    histogram_scores = []
    lbp_scores = []
    hog_scores = []
    sift_scores = []
    combined_scores = []
    
    for result in all_results:
        if result['detailed_scores'] and result['algorithm_analysis']:
            histogram_scores.append(result['algorithm_analysis']['algorithm_scores'].get('histogram', 0))
            lbp_scores.append(result['algorithm_analysis']['algorithm_scores'].get('lbp', 0))
            hog_scores.append(result['algorithm_analysis']['algorithm_scores'].get('hog', 0))
            sift_scores.append(result['algorithm_analysis']['algorithm_scores'].get('sift', 0))
            combined_scores.append(result['similarity_score'])
    
    if histogram_scores:
        algorithm_means = {
            'Histogram': np.mean(histogram_scores),
            'LBP': np.mean(lbp_scores),
            'HOG': np.mean(hog_scores),
            'SIFT': np.mean(sift_scores)
        }
        
        ranked_algorithms = sorted(algorithm_means.items(), key=lambda x: x[1], reverse=True)
        
        print(f"\nFINAL CONCLUSION:")
        print("-" * 40)
        best_algorithm = ranked_algorithms[0]
        worst_algorithm = ranked_algorithms[-1]
        
        print(f"Most effective algorithm: {best_algorithm[0]} ({best_algorithm[1]:.3f})")
        print(f"Least effective algorithm: {worst_algorithm[0]} ({worst_algorithm[1]:.3f})")
        print(f"Overall average score: {np.mean(combined_scores):.3f}")
        print(f"Accuracy rate: {np.mean([r['same_person'] for r in all_results]) * 100:.1f}%")
        
else:
    print("No results to analyze!")

FACIAL RECOGNITION SYSTEM - PAIR COMPARISON
Algorithms used: Haar Features, LBP, HOG, SIFT, Spatial and Frequency Filters
Found 10 original images (A) and 6 transformed images (B)
Pair created: A01.jpg <-> B01.jpg
Pair created: A04.jpg <-> B04.jpg
Pair created: A05.jpg <-> B05.jpg
Pair created: A07.jpg <-> B07.jpg
Pair created: A08.jpg <-> B08.jpg
Pair created: A09.jpg <-> B09.jpg

Found 6 image pairs for processing.
Directory: ./data

[PROCESSING PAIR 1/6]

Processing image pair:
Original Image (A): A01.jpg
Transformed Image (B): B01.jpg
Searching for candidate regions...
Found 0 candidate regions
No face detected. Using central region as fallback.
Total faces detected: 1
Searching for candidate regions...
Found 1 candidate regions
No face detected. Using central region as fallback.
Total faces detected: 1

RESULTS:
Same Person: YES
Confidence: 0.720
------------------------------------------------------------

[PROCESSING PAIR 2/6]

Processing image pair:
Original Image (A): A04.jpg
